### Here, we load in a sample custom function to see how the Mathematica Kernel works

Make sure to update kernelPath to where you have WolframKernel installed

In [1]:
from wolframclient.evaluation import WolframLanguageSession
from wolframclient.language import wl

# Path to the Mathematica kernel
kernelPath = "/Applications/Wolfram.app/Contents/MacOS/WolframKernel"

# Start a Mathematica kernel session
session = WolframLanguageSession(kernel=kernelPath)

# Define the path to the Mathematica script (replace with the correct relative or absolute path)
mathematica_script_path = 'MathematicaScripts/script.wl'

# Load the Mathematica script into the session
session.evaluate(f'Get["{mathematica_script_path}"]')

# Call the function explicitly in the Global context
result = session.evaluate(wl.Global.MyCustomSquareFunction(5))

# Print the result
print(f"The result of MyCustomSquareFunction(5) is: {result}")

# Terminate the session
session.terminate()


The result of MyCustomSquareFunction(5) is: 25


### Perform LLL on a sample function

In [2]:
from wolframclient.evaluation import WolframLanguageSession
from wolframclient.language import wl
from IPython.display import display, Math

# Path to the Mathematica kernel
kernelPath = "/Applications/Wolfram.app/Contents/MacOS/WolframKernel"

# Start a Mathematica kernel session
session = WolframLanguageSession(kernel=kernelPath)

# Define the path to the Mathematica script (replace with the correct relative or absolute path)
mathematica_script_path = 'MathematicaScripts/LLL.wl'

# Load the Mathematica script into the session
session.evaluate(f'Get["{mathematica_script_path}"]')

# Simplified integrand
integrand = wl.Plus(
                4,
                wl.Times(-12, wl.x),
                wl.Times(8, wl.Power(wl.x, 2)),
            )

# Define the basis functions in Python using `wl`
basis_functions = wl.List(
    wl.Divide(1, wl.Power(wl.Pi, 2)),   # 1/π^2
    1,                                  # 1
    wl.Divide(wl.Zeta(3), wl.Power(wl.Pi, 2)),  # Zeta[3]/π^2
    wl.EulerGamma,                      # EulerGamma
    wl.Catalan                         # Catalan
)

# Define WorkingPrecision and PrecisionGoal parameters
working_precision = 10
precision_goal = 10

# Call the EvaluateIntegral function defined in the Mathematica script
result = session.evaluate(
    wl.Global.EvaluateIntegral(integrand, wl.List(wl.x, 0, 1), working_precision, precision_goal)
)

latexInt = session.evaluate(wl.ToString(wl.TeXForm(integrand)))

# Print the result
# Render the LaTeX in Jupyter
display(Math(latexInt))
print(f"The result of the integral is: {result}")

# Call the LatRed function in Mathematica
lllresult = session.evaluate(
    wl.Global.LatRed(basis_functions, result, 10)
)


lllLatex = session.evaluate(wl.ToString(wl.TeXForm(lllresult)))

# Print the result
print(f"LLL result: {lllresult}")
display(Math(lllLatex))

# Call N[lllresult, 40] in Python to get a numerical approximation with 40 digits
numerical_lllresult = session.evaluate(
    wl.N(lllresult, 10)
)

# Print the result
print(f"Numerical LLL result (to 10 digit precision): {numerical_lllresult}")

# Terminate the session
session.terminate()

<IPython.core.display.Math object>

The result of the integral is: 0.66666666666666666668830984884265485462
LLL result: Rational[2, 3]


<IPython.core.display.Math object>

Numerical LLL result (to 10 digit precision): 0.66666666666666666666666666666666666667


### And finally on a more complex function

In [3]:
from wolframclient.language import wlexpr

# Start a Mathematica kernel session
session = WolframLanguageSession(kernel=kernelPath)

# Load the Mathematica script into the session
session.evaluate(f'Get["{mathematica_script_path}"]')

def TestLog(dim):
    # Create the Mathematica expression as a string
    mathematica_expr = f"Total[Table[Log[x + i]^2/i, {{i, 1, {dim}}}]]"
    
    # Pass the expression to the Mathematica session using wlexpr
    return session.evaluate(wlexpr(mathematica_expr))

def LogBasis(xin, dim):
    # Create the Mathematica expression as a string
    mathematica_expr = f"Table[Log[{xin} + i]^2, {{i, 1, {dim}}}]"
    
    # Pass the expression to the Mathematica session using wlexpr
    return session.evaluate(wlexpr(mathematica_expr))

def functlog(xin, dim):
    # Call TestLog and substitute x with xin in the Mathematica expression
    mathematica_expr = f"Total[Table[Log[x + i]^2/i, {{i, 1, {dim}}}]] /. {{x -> {xin}}}"
    
    # Pass the expression to the Mathematica session using wlexpr
    return session.evaluate(wlexpr(mathematica_expr))
    
log = TestLog(5)
logbasis = LogBasis(wl.x, 20)


latexLog = session.evaluate(wl.ToString(wl.TeXForm(log)))
latexLogBasis = session.evaluate(wl.ToString(wl.TeXForm(logbasis)))

# Print the result
# Render the LaTeX in Jupyter
display(Math(latexLog))
display(Math(latexLogBasis))

print("test probe:")

testprobe = [functlog(f"1/{n}", 15) for n in range(3, 8)]

latexProbe = session.evaluate(wl.ToString(wl.TeXForm(testprobe)))
display(Math(latexProbe))

print("basis eval:")

basiseval = [LogBasis(f"1/{n}", 30) for n in range(3, 8)]

latexbasis = session.evaluate(wl.ToString(wl.TeXForm(basiseval)))
display(Math(latexbasis))

print("performed LLL")

arrayLLL = session.evaluate(
    wl.Global.ArrayLatRed(basiseval, testprobe, 70)
)

LatexArrayLLL = session.evaluate(wl.ToString(wl.TeXForm(arrayLLL)))

# Print the result
print(f"Array LLL result:")
display(Math(LatexArrayLLL))


#check if equivalent
mapThread = session.evaluate(wl.MapThread(wl.Times, wl.List(arrayLLL, LogBasis(f"1/{wl.x}", 30))))
sumMap = session.evaluate(wl.Plus(*mapThread))  # This will unpack and sum the elements
mapLatex = session.evaluate(wl.ToString(wl.TeXForm(sumMap)))

# Print the result
print(f"MapThread:")
display(Math(mapLatex))

print("Are equivalent?")
equiv = session.evaluate(sumMap == functlog(f"1/{wl.x}", 15))
print(equiv)

# Terminate the session
session.terminate()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

test probe:


<IPython.core.display.Math object>

basis eval:


...

performed LLL
Array LLL result:


<IPython.core.display.Math object>

MapThread:


<IPython.core.display.Math object>

Are equivalent?
True
